In [1]:
suppressPackageStartupMessages(library(tidyverse))
suppressPackageStartupMessages(library(data.table))


### input files

In [2]:
gwas_covar_f <- '@@@@@@@@@@@@@@@@@@@'


In [3]:
biomarker_f <- '@@@@@@@@@@@@@@@@@@@'


In [4]:
biomarker_trait_list_f <- '@@@@@@@@@@@@@@@@@@@'


In [5]:
master_phe_f <- '@@@@@@@@@@@@@@@@@@@'


### output

In [68]:
phe_info_f <- 'pheno.info.tsv'
phe_file <- 'data/pheno.tsv'


In [10]:
gwas_covar_df <- fread(
    gwas_covar_f,
    select=c('FID', 'IID', 'population', 'split', 'age', 'sex', 'N_CNV', 'LEN_CNV', 'Array', paste0('PC', 1:10)),
    colClasses=c('FID'='character', 'IID'='character')
)


In [12]:
biomarker_trait_list_df <- fread(biomarker_trait_list_f)


In [18]:
biomarker_df <- fread(
    biomarker_f,
    select=c(
        'FID', 'IID', 
        biomarker_trait_list_df %>% 
        filter(annotation != 'Fasting_glucose') %>%
        select(annotation) %>% pull()
    ),
    colClasses=c('FID'='character', 'IID'='character')
)


In [19]:
biomarker_df %>% dim()

[1] 461503     37

In [20]:
blood_ukb_fields <- c(
    # http://biobank.ctsu.ox.ac.uk/crystal/label.cgi?id=100081
30160,
30220,
30150,
30210,
30030,
30020,
30300,
30290,
30280,
30120,
30180,
30050,
30060,
30040,
30100,
30260,
30270,
30130,
30190,
30140,
30200,
30170,
30230,
30080,
30090,
30110,
30010,
30070,
30250,
30240,
30000
)


In [27]:
master_phe_df <- fread(
    master_phe_f,
    select=c(
        'FID', 'IID', 
        paste0('INI', sort(blood_ukb_fields))
    ),
    colClasses=c('FID'='character', 'IID'='character')
)



In [28]:
master_phe_df %>% dim()

[1] 516884     33

In [29]:
library(googlesheets)
gs_auth(token = "@@@@@@@@@@@@@@@@@@@")

file <- '@@@@@@@@@@@@@@@@@@@'
GBE_names_df <- file %>% gs_url() %>% gs_read(ws = 'GBE_names')


Auto-refreshing stale OAuth token.
Sheet-identifying info appears to be a browser URL.
googlesheets will attempt to extract sheet key from the URL.
Putative key: 1gwzS0SVZBSKkkHgsoqB5vHo5JpUeYYz8PK2RWrHEq3A
Sheet successfully identified: "GBE_names"
Accessing worksheet titled 'GBE_names'.
Parsed with column specification:
cols(
  GBE_category = col_character(),
  GBE_ID = col_character(),
  GBE_N = col_double(),
  GBE_NAME = col_character(),
  GBE_short_name = col_character(),
  GBE_short_name_len = col_double()
)


In [56]:
phe_meta_df <- 
GBE_names_df %>% 
filter(GBE_ID %in% paste0('INI', sort(blood_ukb_fields))) %>%
select(GBE_ID, GBE_NAME, GBE_short_name) %>%
rename('pheno_plot' = 'GBE_short_name', 'pheno' = 'GBE_NAME') %>%
bind_rows(
    biomarker_trait_list_df %>% 
    filter(annotation != 'Fasting_glucose') %>%
    rename('pheno' = 'annotation', 'pheno_plot' = 'name') %>%
    mutate(GBE_ID = pheno)
) %>%
select(GBE_ID, pheno, pheno_plot)


In [72]:
phe_meta_df %>% colnames()

[1] "GBE_ID"     "pheno"      "pheno_plot"

In [71]:
phe_meta_df %>% 
rename('#GBE_ID' = 'GBE_ID') %>%
fwrite(phe_info_f, sep='\t')


In [64]:
merged_df <- gwas_covar_df %>%
filter(population == 'white_british') %>%
left_join(
    biomarker_df, by=c('FID', 'IID')
) %>%
left_join(
    master_phe_df, by=c('FID', 'IID')
)


In [67]:
merged_df %>% colnames()

[1] "FID"                         "IID"                        
 [3] "population"                  "split"                      
 [5] "age"                         "sex"                        
 [7] "N_CNV"                       "LEN_CNV"                    
 [9] "Array"                       "PC1"                        
[11] "PC2"                         "PC3"                        
[13] "PC4"                         "PC5"                        
[15] "PC6"                         "PC7"                        
[17] "PC8"                         "PC9"                        
[19] "PC10"                        "Alanine_aminotransferase"   
[21] "Albumin"                     "Alkaline_phosphatase"       
[23] "Apolipoprotein_A"            "Apolipoprotein_B_adjstatins"
[25] "Aspartate_aminotransferase"  "Calcium"                    
[27] "Cholesterol_adjstatins"      "C_reactive_protein"         
[29] "Creatinine"                  "Cystatin_C"                 
[31] "Direct_bilirubin"            "Gamma_glutamyltransferase"  
[33] "Glucose"                     "Glycated_haemoglobin_HbA1c" 
[35] "HDL_cholesterol"             "IGF_1"                      
[37] "LDL_direct_adjstatins"       "Lipoprotein_A"              
[39] "Phosphate"                   "SHBG"                       
[41] "Testosterone"                "Total_bilirubin"            
[43] "Total_protein"               "Triglycerides"              
[45] "Urate"                       "Urea"                       
[47] "Vitamin_D"                   "Creatinine_in_urine"        
[49] "eGFR"                        "Microalbumin_in_urine"      
[51] "Potassium_in_urine"          "Sodium_in_urine"            
[53] "Non_albumin_protein"         "AST_ALT_ratio"              
[55] "INI30000"                    "INI30010"                   
[57] "INI30020"                    "INI30030"                   
[59] "INI30040"                    "INI30050"                   
[61] "INI30060"                    "INI30070"                   
[63] "INI30080"                    "INI30090"                   
[65] "INI30100"                    "INI30110"                   
[67] "INI30120"                    "INI30130"                   
[69] "INI30140"                    "INI30150"                   
[71] "INI30160"                    "INI30170"                   
[73] "INI30180"                    "INI30190"                   
[75] "INI30200"                    "INI30210"                   
[77] "INI30220"                    "INI30230"                   
[79] "INI30240"                    "INI30250"                   
[81] "INI30260"                    "INI30270"                   
[83] "INI30280"                    "INI30290"                   
[85] "INI30300"

In [69]:
merged_df %>% fwrite(phe_file, sep='\t')
